# Git general usage

In this tutorial, we will learn how to use git in general.

## 1. Configure git

To get existing git config value, you can use the below command

```powershell
# general form to get value of a config
git config <config-key>

# for example, get current user name
git config user.name

# if you want to get all config values
git config --list
```


In [1]:
# check if git exists on your shell
! git --version

git version 2.46.0.windows.1


In [ ]:
# get the current user name
! git config user.name

In [ ]:
# get the current user email
! git config user.email

In [2]:
# check existing config
! git config --list

diff.astextplain.textconv=astextplain
filter.lfs.clean=git-lfs clean -- %f
filter.lfs.smudge=git-lfs smudge -- %f
filter.lfs.process=git-lfs filter-process
filter.lfs.required=true
http.sslbackend=openssl
http.sslcainfo=C:/Users/PLIU/AppData/Local/Programs/Git/mingw64/etc/ssl/certs/ca-bundle.crt
core.autocrlf=true
core.fscache=true
core.symlinks=false
pull.rebase=false
credential.helper=manager
credential.https://dev.azure.com.usehttppath=true
init.defaultbranch=master
core.editor="C:\Users\PLIU\AppData\Local\Programs\Microsoft_VS_Code\bin\code" --wait
user.name=pengfei liu
user.email=liu.pengfei@hotmail.fr
core.repositoryformatversion=0
core.filemode=false
core.bare=false
core.logallrefupdates=true
core.symlinks=false
core.ignorecase=true
remote.origin.url=git@github.com:CASD-EU/Seminar_Python_Git.git
remote.origin.fetch=+refs/heads/*:refs/remotes/origin/*
branch.main.remote=origin
branch.main.merge=refs/heads/main


### 1.1 Set config value

There are three levels of configuration:

- `System (set value for all users)`: git config --system
- `Global (set value for current user, and all repos)`: git config --global
- `Local (set value for current user and current repo)`: git config --local

By default, when you set a config value, it uses the **local** level

In [ ]:
# check the default value
! git config init.defaultBranch

In [5]:
# change the default branch name to main
! git config init.defaultBranch main

In [7]:
# check the new values
! git config init.defaultBranch

main


> we can goto another project repo and check the value of `init.defaultBranch`

### 1.2 Delete a config value

We can delete a config value with keyword `--unset`.

```powershell
# the general form is
git config --unset <config-key>

# for example remove user name
git config --unset user.name
```

In [10]:
! git config --global --unset user.name

In [11]:
# now try to get the value
! git config user.name

### 1.3 Minimum config for a new git client

As we explained before, each git commit stores the information of author, so it's important to let `Git` know who you are. For a new git client, the minimum config is to set up the username and email.

```powershell
git config --global --user.name <your-name>
git config --global --user.email <your-email>
```

In [ ]:
! git config --global --user.name pengfei
! git config --global --user.email pengfei.liu@casd.eu

f## 2. Creating a new git repository

Don't use notebook to demonstrate this part. Use the system shell.

```powershell
# create a project folder
mkdir myproject
cd myproject

# Initialize git on the working directory
git init

# you should see below outputs
Initialized empty Git repository in C:/Users/PLIU/Documents/git/myproject/.git/

```

Now you have a **git repository**. All changes inside this repository can be tracked by `git`.

> You can notice the `git init` command generates a folder `.git`. And it has files and folders.
> If you want to know more about git, you can try to understand the functionality of each file and folder in the Appendix chapiter


**Do not edit any file manually inside .git. Any unwanted edition can break your git repo.**



## Appendix

### 1. Understand .git

In general, a `.git` folder has the below contents.

```text
.git/
├── HEAD
├── config
├── description
├── index
├── hooks/
├── info/
├── objects/
├── refs/
│   ├── heads/
│   └── tags/
└── logs/
```

#### 1. HEAD

`HEAD` is a file that points to the current branch. The default value is `ref: refs/heads/master`

If detached, it points directly to a commit hash.

#### 2. config
The `config` file stores the local Git config value for this repo(e.g. remotes, push behavior, fetch rules, etc.).

Below is an example

```text
[core]
        repositoryformatversion = 0
        filemode = false
        bare = false
        logallrefupdates = true
        symlinks = false
        ignorecase = true

```

#### 3. description

The description file stores the default description of the git repo. It's only used by Git web interfaces like GitWeb, but not used by core Git commands.

#### 4. index
The index file is a `binary file` storing what’s staged for the next commit.

#### 5. hooks/

Directory containing sample Git hook scripts.

Used to trigger custom automation (e.g., pre-commit, post-merge).

```
cat .git/hooks/pre-commit

# you should see the below contents
#!/bin/sh
#
# An example hook script to verify what is about to be committed.
# Called by "git commit" with no arguments.  The hook should
# exit with non-zero status after issuing an appropriate message if
# it wants to stop the commit.
#
# To enable this hook, rename this file to "pre-commit".

if git rev-parse --verify HEAD >/dev/null 2>&1
then
        against=HEAD
else
        # Initial commit: diff against an empty tree object
        against=$(git hash-object -t tree /dev/null)
fi

# If you want to allow non-ASCII filenames set this variable to true.
allownonascii=$(git config --type=bool hooks.allownonascii)

# Redirect output to stderr.
exec 1>&2

# Cross platform projects tend to avoid non-ASCII filenames; prevent
# them from being added to the repository. We exploit the fact that the
# printable range starts at the space character and ends with tilde.
if [ "$allownonascii" != "true" ] &&
        # Note that the use of brackets around a tr range is ok here, (it's
        # even required, for portability to Solaris 10's /usr/bin/tr), since
        # the square bracket bytes happen to fall in the designated range.
        test $(git diff-index --cached --name-only --diff-filter=A -z $against |
          LC_ALL=C tr -d '[ -~]\0' | wc -c) != 0
then
        cat <<\EOF
Error: Attempt to add a non-ASCII file name.

This can cause problems if you want to work with people on other platforms.

To be portable it is advisable to rename the file.

If you know what you are doing you can disable this check using:

  git config hooks.allownonascii true
EOF
        exit 1
fi

# If there are whitespace errors, print the offending file names and fail.
exec git diff-index --check --cached $against --

```

> You can notice the script is written in bash, it will not work in windows.
#### 6. info/

The `info/` folder Contains `.git/info/exclude`, which is a local `.gitignore alternative`.

It's useful for ignoring files that shouldn’t be committed but also shouldn’t be in versioned .gitignore.

#### 7. objects/

The **objects** folder is the `core Git storage`. All project versioned data are stored here (e.g. file contents, directories, commit metadata, annotated tag objects)


> They are organized by SHA-1 prefix:

```text
.git/objects/ab/cd1234…  ← object hash abcd1234…
```

#### 8. refs/
The `refs` folder stores references (pointers) to commits:

```text
refs/heads/ → local branches

refs/tags/ → tags

refs/remotes/ → remote-tracking branches (after remote added)

```

#### 9. logs/
The `logs` folder keeps history of HEAD and branch movements.

Enables powerful undo via git reflog.

```powershell
git reflog

# output example
02456da (HEAD -> master) HEAD@{0}: commit (initial): first commits

```